In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
root = '/content/drive/MyDrive/CS7643/Project'

In [13]:
import os
os.chdir(root + '/data/tokenizer')
%pwd

'/content/drive/MyDrive/CS7643/Project/data/tokenizer'

Ref: https://towardsdatascience.com/how-to-build-a-wordpiece-tokenizer-for-bert-f505d97dddbb

In [14]:
!pip install datasets
!pip install transformers
!pip install tokenizers

     |████████████████████████████████| 325 kB 8.6 MB/s 
     |████████████████████████████████| 212 kB 57.5 MB/s 
     |████████████████████████████████| 1.1 MB 56.9 MB/s 
     |████████████████████████████████| 77 kB 7.6 MB/s 
     |████████████████████████████████| 136 kB 69.7 MB/s 
     |████████████████████████████████| 127 kB 65.4 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 73.1 MB/s 
     |████████████████████████████████| 271 kB 72.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 4.0 MB 8.3 MB/s 
     |████████████████████████████████| 895 kB 42.2 MB/s 
     |████████████████████████████████| 6.5 MB 52.7 MB/s 
     |████████████████████████████████| 596 kB 65.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [15]:
from datasets import load_dataset

Generate Tokenizer from News dataset

In [16]:
dataset = load_dataset('ag_news')

Using custom data configuration default


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 120000
})

In [19]:
import os
isExist = os.path.exists('./ag-news')
if not isExist:
  os.mkdir('./ag-news')


In [20]:
from tqdm.auto import tqdm  # for our loading bar

text_data = []
file_count = 0

for sample in tqdm(dataset['train']):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample['text'].replace('\n', '\s')
    text_data.append(sample)
    if len(text_data) == 5_000:
        # once we hit the 5K mark, save to file
        with open(f'./ag-news/file_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 5K chunks, we may have leftover samples, we save those now too
with open(f'./ag-news/file_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/120000 [00:00<?, ?it/s]

In [21]:
from pathlib import Path
paths = [str(x) for x in Path('./ag-news').glob('**/*.txt')]
paths[:5]

['ag-news/file_0.txt',
 'ag-news/file_1.txt',
 'ag-news/file_2.txt',
 'ag-news/file_3.txt',
 'ag-news/file_4.txt']

In [22]:
len(paths)

25

In [23]:
# !pip install tokenizers
from tokenizers import BertWordPieceTokenizer

# initialize
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)
# and train
tokenizer.train(files=paths, vocab_size=30_000, min_frequency=2,
                limit_alphabet=1000, wordpieces_prefix='##',
                special_tokens=[
                    '[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'])

In [24]:
isExist = os.path.exists('./news_tokenizer')
if not isExist:
  os.mkdir('./news_tokenizer')
tokenizer.save_model('./news_tokenizer')

['./news_tokenizer/vocab.txt']

In [25]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('./news_tokenizer')

Generate Tokenizer from our dataset

In [27]:
os.chdir(root)
%pwd

'/content/drive/MyDrive/CS7643/Project'

In [28]:
# All data will be loaded together unless data_files is specified
dataset = load_dataset('data', data_files ={'train':'train.csv','test':'test.csv'})
dataset = dataset.rename_column("Unnamed: 0", 'id')

Using custom data configuration data-cf9ebd5eebbe0a50


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/data-cf9ebd5eebbe0a50/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
dataset['train']

Dataset({
    features: ['id', 'text', 'label'],
    num_rows: 35918
})

In [30]:
os.chdir(root + '/data/tokenizer')
%pwd

'/content/drive/MyDrive/CS7643/Project/data/tokenizer'

In [32]:
import os
isExist = os.path.exists('./fake-news')
if not isExist:
  os.mkdir('./fake-news')


In [33]:
from tqdm.auto import tqdm  # for our loading bar

text_data = []
file_count = 0

for sample in tqdm(dataset['train']):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample['text'].replace('\n', '\s')
    text_data.append(sample)
    if len(text_data) == 5_000:
        # once we hit the 5K mark, save to file
        with open(f'./fake-news/file_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 5K chunks, we may have leftover samples, we save those now too
with open(f'./fake-news/file_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/35918 [00:00<?, ?it/s]

In [34]:
from pathlib import Path
paths = [str(x) for x in Path('./fake-news').glob('**/*.txt')]
paths[:5]

['fake-news/file_0.txt',
 'fake-news/file_1.txt',
 'fake-news/file_2.txt',
 'fake-news/file_3.txt',
 'fake-news/file_4.txt']

In [35]:
# !pip install tokenizers
from tokenizers import BertWordPieceTokenizer

# initialize
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)
# and train
tokenizer.train(files=paths, vocab_size=30_000, min_frequency=2,
                limit_alphabet=1000, wordpieces_prefix='##',
                special_tokens=[
                    '[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'])

In [36]:
isExists = os.path.exists('./fake_tokenizer')
if not isExists:
  os.mkdir('./fake_tokenizer')
tokenizer.save_model('./fake_tokenizer')

['./fake_tokenizer/vocab.txt']

In [37]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('./fake_tokenizer')